In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from collections import Counter
import numpy as np
from numpy import loadtxt
import tensorflow as tf

f = h5py.File('mouse1sample1.hdf5','r')

In [ ]:
annotated_cells = []
annotations = []
for cell_id in f['cells']:
    cell = f['cells'][cell_id]
    ann = dict(cell.attrs)['annotation']
    if ann != 'unannotated':
        annotated_cells.append(cell_id)
        annotations.append(ann)

In [ ]:
annotations_count = Counter(annotations)
print("Number of Annotated Samples:", len(annotated_cells))
print("Num of Cell Types:", len(annotations_count))
print("Max Number of Samples per Cell Type:", max(annotations_count.values()))
print("Min Number of Samples per Cell Type:", min(annotations_count.values()))
print("Average Number of Samples per Cell Type:", sum(annotations_count.values())/93)

#graph of num of samples across cell-types 
plt.bar(np.arange(0,93), annotations_count.values())

In [ ]:
# # creating gene expression table -- time intensive computation
# df = pd.DataFrame()
# for cell_id in annotated_cells:
#     cell = f['cells'][cell_id]
#     gene_counts = []
#     for z in cell.attrs['zslices']: # for each z-slice corresponding to cell
#         spot_genes = cell['spot_genes'][z] # get spot_genes
#         for gene in spot_genes: # iterate over spot_genes 
#             gene_counts.append(gene.decode())
#     gene_counts = dict(Counter(gene_counts)) # get count of all genes for cell
#     df = df.append(gene_counts, ignore_index=True) # add row to pd dataframe
# df = df.fillna(0)
# df.to_csv('geneExpression.csv', sep='\t') 

In [ ]:
# creating one-hot coded vectors for cell-type (labels for multi)
annotation_indices = {k: v for v, k in enumerate(list(annotations_count.keys()))}
labels = np.zeros((len(annotated_cells), len(annotation_indices)))
for index in range(0, len(annotated_cells)):
    cell = f['cells'][annotated_cells[index]]
    ann = dict(cell.attrs)['annotation']
    arr_index = annotation_indices[ann]
    labels[index][arr_index] = 1

In [ ]:
dataset = loadtxt('geneExpression.csv', delimiter=',')
print(dataset.shape)

In [ ]:
# normalize + scale gene expression data + remove outliers (code from ACTINN paper)
total_set = np.divide(dataset, np.sum(dataset, axis=0, keepdims=True)) * 10000
total_set = np.log2(total_set+1)
expr = np.sum(total_set, axis=1)
total_set = total_set[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
total_labels = labels[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
cv = np.std(total_set, axis=1) / np.mean(total_set, axis=1)
total_set = total_set[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]
total_labels = total_labels[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]

dataset=total_set

In [ ]:
from sklearn.model_selection import train_test_split
train, test, train_y, test_y = train_test_split(dataset, total_labels, test_size=0.3)

test, val, test_y, val_y = train_test_split(test, test_y, test_size=0.5)
print(train.shape, train_y.shape)
print(val.shape, val_y.shape)
print(test.shape, test_y.shape)

In [ ]:
# followed model parameters + layers according to ACTINN paper 
model = tf.keras.models.Sequential([tf.keras.layers.Input(252),
                                    tf.keras.layers.Dense(100, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(50, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(25, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(93, activation=tf.nn.softmax)])
lr= tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=100000,
    decay_rate=0.95,
    staircase=True)
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
model.summary()
model.fit(train, train_y, batch_size=128, epochs=50, validation_data=(val, val_y))

In [ ]:
model.evaluate(test, test_y)

In [ ]:
y_pred = model.predict(test)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_y, axis=1)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)